## Local embeddings modeling notebook

In [12]:
%load_ext autoreload
%autoreload 2

from src.ui import GeoVibes

vibes = GeoVibes.from_config('config/ui_config.json')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Initializing GeoLabeler...
Building UI...


HTML(value='\n        <style>\n        .widget-toggle-buttons button:nth-child(1).mod-active {\n            ba…